In [2]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u

In [60]:
det_cats = {
    'pz14' : pd.read_csv('/home/rt2122/Data/scans_old/scans_extended/connected/ep14_thr0.1.csv'),
    'pz25' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz25_thr0.1_step8.csv'),
    'pz_act14' : pd.read_csv(
    '/home/rt2122/Data/scans_old/scans_extended/connected/full_act_cut_ep14_thr0.1_step8.csv'),
    'pz_act20' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv'),
}

In [4]:
for name in det_cats:
    det_cats[name] = det_cats[name][det_cats[name]['status']!='fn']
    det_cats[name].index = np.arange(len(det_cats[name]))

In [56]:
def merge_cats(cat1, cat2, merge_dist=5/60): 
    import numpy as np
    import pandas as pd
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    sc1 = SkyCoord(ra=np.array(cat1['RA'])*u.degree, dec=np.array(cat1['DEC'])*u.degree, frame='icrs')
    sc2 = SkyCoord(ra=np.array(cat2['RA'])*u.degree, dec=np.array(cat2['DEC'])*u.degree, frame='icrs')
    
    idx, d2d, _ = sc1.match_to_catalog_sky(sc2)
    matched = d2d.degree <= merge_dist
    
    cat_matched = cat1.iloc[matched].copy()
    cat1_uniq = cat1.iloc[matched == False].copy()
    cat2_uniq = cat2.iloc[np.array(idx[matched == False])].copy()
    
    return pd.concat([cat_matched, cat1_uniq, cat2_uniq], ignore_index=True)

In [61]:
res_cat = det_cats['pz14']
for name in det_cats:
    if name == 'pz14':
        continue
    res_cat = merge_cats(res_cat, det_cats[name])

In [62]:
res_cat.shape

(68699, 12)

In [65]:
res_cat

,RA,DEC,area,min_rad,max_rad,mean_rad,min_pred,max_pred,tRA,tDEC,status,catalog
0,278.763931,30.386470,19.0,1.211624,3.403813,2.555837,0.134940,0.574060,NaN,NaN,fp,NaN
1,278.255559,30.812475,12.0,0.869420,3.267161,2.120304,0.110287,0.304136,NaN,NaN,fp,NaN
2,278.645841,26.214733,29.0,1.345853,4.610579,3.172637,0.102232,0.322857,NaN,NaN,fp,NaN
3,280.821471,14.893659,13.0,0.890057,3.179697,2.187293,0.100190,0.143267,NaN,NaN,fp,NaN
4,276.381603,27.739437,28.0,1.454716,3.958601,3.061761,0.140074,0.884521,NaN,NaN,fp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
68694,164.271542,-78.390426,13.0,0.639551,3.539868,2.213535,0.101869,0.533667,NaN,NaN,fp,NaN
68695,164.271542,-78.390426,13.0,0.639551,3.539868,2.213535,0.101869,0.533667,NaN,NaN,fp,NaN
68696,222.000046,-60.411481,14.0,1.063246,3.877776,2.201008,0.101978,0.332434,NaN,NaN,fp,NaN
68697,164.271542,-78.390426,13.0,0.639551,3.539868,2.213535,0.101869,0.533667,NaN,NaN,fp,NaN


In [66]:
res_cat.drop(['tRA', 'tDEC', 'status', 'catalog'], axis='columns', inplace=True)
res_cat

,RA,DEC,area,min_rad,max_rad,mean_rad,min_pred,max_pred
0,278.763931,30.386470,19.0,1.211624,3.403813,2.555837,0.134940,0.574060
1,278.255559,30.812475,12.0,0.869420,3.267161,2.120304,0.110287,0.304136
2,278.645841,26.214733,29.0,1.345853,4.610579,3.172637,0.102232,0.322857
3,280.821471,14.893659,13.0,0.890057,3.179697,2.187293,0.100190,0.143267
4,276.381603,27.739437,28.0,1.454716,3.958601,3.061761,0.140074,0.884521
...,...,...,...,...,...,...,...,...
68694,164.271542,-78.390426,13.0,0.639551,3.539868,2.213535,0.101869,0.533667
68695,164.271542,-78.390426,13.0,0.639551,3.539868,2.213535,0.101869,0.533667
68696,222.000046,-60.411481,14.0,1.063246,3.877776,2.201008,0.101978,0.332434
68697,164.271542,-78.390426,13.0,0.639551,3.539868,2.213535,0.101869,0.533667


In [67]:
res_cat.to_csv('/home/rt2122/Data/detected_cats/merged.csv', index=False)